In [80]:
import numpy as np
import pandas as pd
import psycopg2
import time
import datetime
import sys
from constants import DB_NAME, DB_HOST, DB_PASS, DB_PORT, DB_USER, DB_TIME

news = 'mariana-ferrer'

In [81]:
news_insert_template = """INSERT INTO news(news_id,title,link,deleted) VALUES({news_id},'{title}','{link}',false);
"""
commnets_insert_template = """INSERT INTO comments(comment_id,content,likes,dislikes,reply_to,news_id,deleted) VALUES({comment_id},'{content}',{likes},{dislikes},'{reply_to}',{news_id},false);
"""
query = ''

In [82]:
def init_db_connection():
    today = datetime.datetime.now()

    try:  # connect to postgres
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            host=DB_HOST,
            port=DB_PORT,
            password=DB_PASS,
            connect_timeout=DB_TIME
        )
        tm = time.time()
        return conn
        print('Connected to database: [%s] @ %d seconds' %
            (DB_HOST, time.time() - tm))
    except Exception as e:
        print('Cannot connect to Postgres [%s]' % DB_HOST)
        print(str(e))
        exit(1)

In [83]:
col_names = ['id',
             'content',
             'likes',
             'dislikes',
             'reply_to',
             'news_id',
             'news_title',
             'news_link']
data = pd.read_csv('./collected-data/{}-comments.csv'.format(news),sep=';',names=col_names)

In [84]:
data.head()

,id,content,likes,dislikes,reply_to,news_id,news_title,news_link
0,a1a52079-b54e-4780-a706-b557d662f9ad,sou a favor dela.,1,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Caso mariana ferrer deputadas apresentam proje...,https://g1.globo.com/politica/noticia/2020/11/...
1,6500830d-b4b8-41b0-8062-dee5b365d21f,Essa garota nada mais é que uma gol-p-is-t-a d...,2,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Caso mariana ferrer deputadas apresentam proje...,https://g1.globo.com/politica/noticia/2020/11/...
2,8865926f-1ed5-4128-8b32-13e07ffbf2d8,Tudo isso acontecendo porque o LERDO do juiz a...,2,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Caso mariana ferrer deputadas apresentam proje...,https://g1.globo.com/politica/noticia/2020/11/...
3,31047384-98ac-46c4-9ce6-d16cfb2b2e33,"Elas nem assistiram o vídeo na íntegra, e já q...",2,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Caso mariana ferrer deputadas apresentam proje...,https://g1.globo.com/politica/noticia/2020/11/...
4,b773d2b0-a1c6-4bf2-ad17-3053ddae98e1,Fosse esse um país decente e não precisaríamos...,0,0,NaN,a1f4c69f-7d0f-427f-96d8-6ae04102450a,Caso mariana ferrer deputadas apresentam proje...,https://g1.globo.com/politica/noticia/2020/11/...


In [ ]:
conn = init_db_connection()
cursor = conn.cursor()

news = data[['news_title','news_link']].drop_duplicates()
news_titles = list(news['news_title'].drop_duplicates())
news_titles.remove(np.nan)

cursor.execute('select max(news_id) from news;')
news_id = int(cursor.fetchone()[0])
cursor.execute('select max(comment_id) from comments;')
comment_id = int(cursor.fetchone()[0])
i=0

for new in news_titles:
    news_id +=1
    comments = data[data['news_title']==new]
    news_insert = news_insert_template.format(
        news_id=news_id,
        title=str(news[news['news_title']==new]['news_title'].max()),
        link=str(news[news['news_title']==new]['news_link'].max())
    )
    cursor.execute(news_insert)
    query += news_insert
    news_comment = comments[comments['news_title']==new]
    for index, row in news_comment.iterrows():
        comment_id+=1
        comment_insert = commnets_insert_template.format(
            comment_id=comment_id,
            content=row['content'],
            likes=row['likes'],
            dislikes=row['dislikes'],
            reply_to=row['reply_to'],
            news_id=news_id
        )
        comment_insert = comment_insert.replace("'nan'", 'NULL')
        i+=1
        query += comment_insert
        cursor.execute(comment_insert)
cursor.close()
conn.commit()
conn.close()

In [40]:
comment_insert

"INSERT INTO comments(comment_id,content,likes,dislikes,reply_to,news_id,deleted) VALUES(3630,'Cuida da vida dela. Não foi ela que queria impedir o aborto em uma menina de 11 anos? Aff',4,0,NULL,26,false);\n"

In [41]:
news_id

26

In [92]:
news_titles

['Caso mariana ferrer deputadas apresentam projeto para tornar crime a violencia instituciona']